# Basics & Prereqs (run once)

If you don't already have the downloaded dependencies; if you don't have TheMovieDB data indexed run this

In [1]:
from ltr.client.elastic_client import ElasticClient
client = ElasticClient()

from ltr import download, index
download.run(); index.rebuild_tmdb(client)

GET http://es-learn-to-rank.labs.o19s.com/tmdb.json
GET http://es-learn-to-rank.labs.o19s.com/RankyMcRankFace.jar
GET http://es-learn-to-rank.labs.o19s.com/title_judgments.txt
Done.
RUNNING INDEXING...
DO IT!
Deleted index tmdb [Status: 200]
Created index tmdb [Status: 200]
Indexing 27846 documents
Black Mirror: White Christmas added to tmdb
The Brave-Hearted Will Take the Bride added to tmdb
The Shawshank Redemption added to tmdb
Your Name. added to tmdb
The Godfather added to tmdb
Dangal added to tmdb
Whiplash added to tmdb
Schindler's List added to tmdb
Spirited Away added to tmdb
The Godfather: Part II added to tmdb
Sherlock: The Final Problem added to tmdb
Tokyo Story added to tmdb
Fight Club added to tmdb
Pulp Fiction added to tmdb
Life Is Beautiful added to tmdb
The Dark Knight added to tmdb
Woman in the Dunes added to tmdb
GoodFellas added to tmdb
One Flew Over the Cuckoo's Nest added to tmdb
Forrest Gump added to tmdb
Seven Samurai added to tmdb
Piper added to tmdb
The Green M

Downfall added to tmdb
Lilya 4-Ever added to tmdb
Tokyo Godfathers added to tmdb
Children of Heaven added to tmdb
Straight Outta Compton added to tmdb
Kagemusha added to tmdb
Freaks added to tmdb
Carlito's Way added to tmdb
Lifted added to tmdb
The Gold Rush added to tmdb
Train to Busan added to tmdb
It Happened One Night added to tmdb
Batman Beyond: Return of the Joker added to tmdb
The Red Balloon added to tmdb
The Manchurian Candidate added to tmdb
Nothing Left to Do but Cry added to tmdb
The Martian added to tmdb
The City of Lost Children added to tmdb
Fallen Angels added to tmdb
Day & Night added to tmdb
Duck Soup added to tmdb
The Raid 2 added to tmdb
Batman: The Dark Knight Returns, Part 1 added to tmdb
Departures added to tmdb
A Beautiful Mind added to tmdb
The Notebook added to tmdb
Duck, You Sucker added to tmdb
Kingsman: The Secret Service added to tmdb
Barry Lyndon added to tmdb
Cool Hand Luke added to tmdb
Stalag 17 added to tmdb
Infernal Affairs added to tmdb
For the Bird

Evangelion: 1.0: You Are (Not) Alone added to tmdb
Dawn of the Planet of the Apes added to tmdb
Star Trek added to tmdb
The Kite Runner added to tmdb
Harry Potter and the Half-Blood Prince added to tmdb
Inception: The Cobol Job added to tmdb
Fist of Fury added to tmdb
Adaptation. added to tmdb
Birdman added to tmdb
Tombstone added to tmdb
Stranger Than Paradise added to tmdb
Nebraska added to tmdb
Sleuth added to tmdb
Mysterious Skin added to tmdb
Sling Blade added to tmdb
Synecdoche, New York added to tmdb
Star Trek Into Darkness added to tmdb
Contempt added to tmdb
The Warriors added to tmdb
Clerks added to tmdb
The Incredibles added to tmdb
I Can Quit Whenever I Want added to tmdb
The Devils added to tmdb
Dragon Ball Z: Fusion Reborn added to tmdb
Happiness added to tmdb
Anastasia added to tmdb
High Plains Drifter added to tmdb
Three Steps Above Heaven added to tmdb
Underground added to tmdb
Rogue One: A Star Wars Story added to tmdb
Colonia added to tmdb
This Is England added to tm

Falling Down added to tmdb
Kung Fu Hustle added to tmdb
Suspicion added to tmdb
Frances Ha added to tmdb
Menace II Society added to tmdb
The Cat Returns added to tmdb
The Ladykillers added to tmdb
Raising Arizona added to tmdb
American Splendor added to tmdb
Cape Fear added to tmdb
Forbidden Planet added to tmdb
Pelé: Birth of a Legend added to tmdb
The Odyssey added to tmdb
The Blind Side added to tmdb
The Sandlot added to tmdb
The Punisher: Dirty Laundry added to tmdb
The Judge added to tmdb
Somewhere in Time added to tmdb
Sarah's Key added to tmdb
Black Hawk Down added to tmdb
Minions: Banana added to tmdb
From Here to Eternity added to tmdb
Invasion of the Body Snatchers added to tmdb
Interview with the Vampire added to tmdb
Garden State added to tmdb
25th Hour added to tmdb
Eddie the Eagle added to tmdb
Love & Basketball added to tmdb
Chinese Take-Out added to tmdb
The Butler added to tmdb
The Motorcycle Diaries added to tmdb
Rang De Basanti added to tmdb
Bring Me the Head of Alfr

Friday Night Lights added to tmdb
Rise of the Guardians added to tmdb
Juno added to tmdb
Cape Fear added to tmdb
Tracks added to tmdb
Fahrenheit 451 added to tmdb
The Land Before Time added to tmdb
Bridge to Terabithia added to tmdb
Hotel Chevalier added to tmdb
22 Jump Street added to tmdb
Scream added to tmdb
The Age of Innocence added to tmdb
Broken Embraces added to tmdb
Suite Française added to tmdb
WarGames added to tmdb
Conviction added to tmdb
Dead Ringers added to tmdb
Becoming Jane added to tmdb
Dogtooth added to tmdb
Wall Street added to tmdb
Maleficent added to tmdb
Concussion added to tmdb
The Assassination of Jesse James by the Coward Robert Ford added to tmdb
Dirty Dancing added to tmdb
Sleep Tight added to tmdb
Take the Money and Run added to tmdb
The Baader Meinhof Complex added to tmdb
Candy added to tmdb
The Wind That Shakes the Barley added to tmdb
Selena added to tmdb
The Phantom of the Opera added to tmdb
Shallow Grave added to tmdb
The Lincoln Lawyer added to tmd

...

Dealer added to tmdb
The Central Park Five added to tmdb
Far Out Isn't Far Enough: The Tomi Ungerer Story added to tmdb
How to Make Money Selling Drugs added to tmdb
Mea Maxima Culpa: Silence in the House of God added to tmdb
More Than Honey added to tmdb
Hyenas added to tmdb
Olivier, Olivier added to tmdb
The Package added to tmdb
Where a Good Man Goes added to tmdb
The Chosen One added to tmdb
The Last Ride added to tmdb
Diario de un skin added to tmdb
Grave Secrets added to tmdb
Metal Gear Solid: Philanthropy added to tmdb
Miracle Run added to tmdb
Dolls and Angels added to tmdb
The Dancer Upstairs added to tmdb
High on Crack Street: Lost Lives in Lowell added to tmdb
A Private Function added to tmdb
The Thieves added to tmdb
The Solid Gold Cadillac added to tmdb
The Hit List added to tmdb
Alphabet City added to tmdb
Ready to Rumble added to tmdb
Closet Land added to tmdb
November added to tmdb
Parade added to tmdb
Bad Ass 2: Bad Asses added to tmdb
America: Imagine the World Withou

Judy Moody and the Not Bummer Summer added to tmdb
Spare Bed-Room added to tmdb
The Alarmist added to tmdb
Girl 27 added to tmdb
Stanley Kubrick's Boxes added to tmdb
Sprung added to tmdb
Salut cousin ! added to tmdb
Babysitter Wanted added to tmdb
The Other Son added to tmdb
Loser Takes All! added to tmdb
The Living Idol added to tmdb
Crockdale added to tmdb
Finisterrae added to tmdb
Men Seeking Women added to tmdb
Wicked Blood added to tmdb
The Sleeping Car added to tmdb
Allotment Wives added to tmdb
Lavoura Arcaica added to tmdb
Rhythm of a Crime added to tmdb
The Almost Man added to tmdb
L.A. Zombie added to tmdb
BookWars added to tmdb
Gamera the Brave added to tmdb
Faat Kiné added to tmdb
Wings added to tmdb
The Galaxy Invader added to tmdb
Innocence added to tmdb
Joe Gould's Secret added to tmdb
Last Resort added to tmdb
Time of Favor added to tmdb
The Crowd Roars added to tmdb
The 10th District Court: Moments of Trial added to tmdb
The Gaucho added to tmdb
Five Graves to Cairo a

Impudent Girl added to tmdb
Ballerina added to tmdb
Les Misérables added to tmdb
Beware the Moon: Remembering 'An American Werewolf in London' added to tmdb
Johnny Belinda added to tmdb
Born to Win added to tmdb
The Iran Job added to tmdb
009 RE：CYBORG added to tmdb
Le Poulpe added to tmdb
The Clock added to tmdb
Patton Oswalt: Finest Hour added to tmdb
Somewhere in the City added to tmdb
High School Hellcats added to tmdb
Breaking Point added to tmdb
Only God Knows added to tmdb
Until the End of the World added to tmdb
The Earthling added to tmdb
The Young Philadelphians added to tmdb
Belle of the Nineties added to tmdb
Happy Go Lovely added to tmdb
Four More Years added to tmdb
Neon Flesh added to tmdb
Almost 18 added to tmdb
Dead Meat added to tmdb
High School Confidential! added to tmdb
Run If You Can added to tmdb
Agnosia added to tmdb
Earth Days added to tmdb
The Legend of Mor'du added to tmdb
The Case Against 8 added to tmdb
Bikes vs Cars added to tmdb
Murderous Maids added to t

The Crystal Ball added to tmdb
Modern Romance added to tmdb
A Stranger in Town added to tmdb
Heaven added to tmdb
Cooking up dreams added to tmdb
Seal Team Eight: Behind Enemy Lines added to tmdb
My Man and I added to tmdb
Vice Squad added to tmdb
A Story of Children and Film added to tmdb
The Good Humor Man added to tmdb
Calmos added to tmdb
The Incite Mill - 7 Day Death Game added to tmdb
Paying the Price: Killing the Children of Iraq added to tmdb
Chico & Rita added to tmdb
Jo Jo Dancer, Your Life Is Calling added to tmdb
The Turn of the Screw added to tmdb
Bullets Don't Argue added to tmdb
Zatôichi Challenged added to tmdb
The Broadway Melody added to tmdb
Gitmo added to tmdb
Flash Gordon: Trip to Mars added to tmdb
Leap Year added to tmdb
The White Sheik added to tmdb
The Last of the High Kings added to tmdb
Hit Lady added to tmdb
Lovely to Look At added to tmdb
Joyful Noise added to tmdb
Project Nim added to tmdb
Louis Theroux: The Most Hated Family in America added to tmdb
Terry

Two for the Seesaw added to tmdb
Mr. Average added to tmdb
The Sword and the Rose added to tmdb
London After Midnight added to tmdb
Seconds Apart added to tmdb
The Debutantes added to tmdb
Thirst added to tmdb
Broadway Serenade added to tmdb
Greyfriars Bobby: The True Story of a Dog added to tmdb
The Legend of Lobo added to tmdb
The Fighting Prince of Donegal added to tmdb
Ohyaku: The Female Demon added to tmdb
The One and Only, Genuine, Original Family Band added to tmdb
The Story of Three Loves added to tmdb
Stormy Waters added to tmdb
A Sixth Part of the World added to tmdb
I Can't Sleep added to tmdb
Cold Water added to tmdb
Finding a Family added to tmdb
Consuming Spirits added to tmdb
The Sugar Curtain added to tmdb
Gulliver's Travels added to tmdb
Tora-san's Rise and Fall added to tmdb
Le Nouveau Jean-Claude added to tmdb
Murder à la Mod added to tmdb
Young Cassidy added to tmdb
Mary added to tmdb
The Two Jakes added to tmdb
Candyman: Farewell to the Flesh added to tmdb
Echoes f

## Create Elastic Client

In [2]:
from ltr.client.elastic_client import ElasticClient
client = ElasticClient()

# Our Task: Optimizing "Drama" and "Science Fiction" queries

In this example we have two user queries

- Drama
- Science Fiction

And we want to train a model to return the best movies for these movies when a user types them into our search bar.

We learn through analysis that searchers prefer newer science fiction, but older drama. Like a lot of search relevance problems, two queries need to be optimized in *different* directions

### Synthetic Judgment List Generation

To setup this example, we'll generate a judgment list that rewards new science fiction movies as more relevant; and old drama movies as relevant.

In [3]:
from ltr import date_genre_judgments
judgments = date_genre_judgments.buildJudgments(client, judgmentsFile='data/genre_by_date_judgments.txt')

Generating judgments for scifi & drama movies
Searching tmdb - {'query': {'match_al [Status: 200]
Done


In [2]:
# Uncomment this line to see the judgments
# 
# for judgment in judgments:
#    print(judgment.toRanklibFormat())

### Feature selection should be *easy!*

Notice we have 4 proposed features, that seem like they should work! This should be a piece of cake...

1. Release Year of a movie `release_year` - feature ID 1
2. Is the movie Science Fiction `is_scifi` - feature ID 2
3. Is the movie Drama `is_drama` - feature ID 3
4. Does the search term match the genre field `is_genre_match` - feature ID 4


In [4]:
config = {
    "featureset": {
            "features": [
            {
                "name": "release_year",
                "params": [],
                "template": {
                    "function_score": {
                        "field_value_factor": {
                        "field": "release_year",
                        "missing": 2000
                    },
                    "query": { "match_all": {} }
                }
            }
            },
             {
                "name": "is_sci_fi",
                "params": [],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match_phrase": {"genres": "Science Fiction"}
                        },
                        "boost": 10.0                    }
            }
            },
             {
                "name": "is_drama",
                "params": [],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match_phrase": {"genres": "Drama"}
                        },
                        "boost": 4.0                    }
                }
            },
             {
                "name": "is_genre_match",
                "params": ["keywords"],
                "template": {
                    "constant_score": {
                        "filter": {
                            "match_phrase": {"genres": "{{keywords}}"}
                        },
                        "boost": 100.0
                    }
                }
            }
    ]
    },
    "validation": {
       "params": {
           "keywords": "Science Fiction"
       },
       "index": "tmdb"
    }
}

from ltr.setup import setup
setup(client, config=config, featureset='genre')

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create genre feature set [Status: 201]


### Log from search engine -> to training set

Each feature is a query to be scored against the judgment list

In [8]:
from ltr.log import judgments_to_training_set
trainingSet = judgments_to_training_set(client,
                                        judgmentInFile='data/genre_by_date_judgments.txt', 
                                        trainingOutFile='data/genre_by_date_judgments_train.txt', 
                                        featureSet='genre')

Recognizing 2 queries...
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for Science Fiction (0/2)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for Drama (1/2)


### Training - Guaraneed Perfect Search Results!

We'll train a LambdaMART model against this training data.

In [6]:
from ltr.train import train
trainLog = train(client,
                 trainingInFile='data/genre_by_date_judgments_train.txt',
                 metric2t='NDCG@10',
                 featureSet='genre',
                 modelName='genre')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
print("Perfect NDCG! {}".format(trainLog.rounds[-1]))

Running java -jar data/RankyMcRankFace.jar -ranker 6 -metric2t NDCG@10 -tree 100 -train data/genre_by_date_judgments_train.txt -save data/genre_model.txt
DONE
Delete model genre: 404
Created Model genre [Status: 201]

Impact of each feature on the model
1 - 84672139.80027786
2 - 57048.53838040621
3 - 15085.984030299318
4 - 0.0
Perfect NDCG! 1.0


### But this search sucks!
Try searches for "Science Fiction" and "Drama"

In [11]:
from ltr.search import search
search(client, keywords="Science Fiction", modelName="genre")

ImportError: cannot import name 'search'

### Why didn't it work!?!? Training data

1. Examine the training data, do we cover every example of a BAD result
2. Examine the feature impacts, do any of the features the model uses even USE the keywords?

### Ranklib only sees the data you give it, we don't have good enough coverage

You need to have feature coverage, especially over negative examples. Most documents in the index are negative! 

One trick commonly used is to treat other queries positive results as this queries negative results. Indeed what we're missing here are negative examples for "Science Fiction" that are not science fiction movies. A glaring omission, we'll handle now... With the `autoNegate` flag, we'll add additional negative examples to the judgment list

In [9]:
from ltr import date_genre_judgments
from ltr.log import judgments_to_training_set
date_genre_judgments.buildJudgments(client,
                                    judgmentsFile='data/genre_by_date_judgments.txt',
                                    autoNegate=True)

judgments_to_training_set(client,
                          judgmentInFile='data/genre_by_date_judgments.txt', 
                          trainingOutFile='data/genre_by_date_judgments_train.txt', 
                          featureSet='genre')

from ltr.train import train
trainLog = train(client,
                 trainingInFile='data/genre_by_date_judgments_train.txt',
                 metric2t='NDCG@10',
                 featureSet='genre',
                 modelName='genre')

print()
print("Impact of each feature on the model")
for ftrId, impact in trainLog.impacts.items():
    print("{} - {}".format(ftrId, impact))
    
print("Perfect NDCG! {}".format(trainLog.rounds[-1]))

Generating judgments for scifi & drama movies
Searching tmdb - {'query': {'match_al [Status: 200]
Done
Recognizing 2 queries...
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for Science Fiction (0/2)
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
Searching tmdb - [{'terms': {'_id': [ [Status: 200]
REBUILDING TRAINING DATA for Drama (1/2)
Running java -jar data/RankyMcRankFace.jar -ranker 6 -metric2t NDCG@10 -tree 100 -train data/genre_by_date_judgments_train.txt -save data/genre_model.txt
DO

### Now try those queries...

Replace keywords below with 'science fiction' or 'drama' and see how it works

In [10]:
from ltr.search import search
search(client, keywords="Drama", modelName="genre")

ImportError: cannot import name 'search'

### The next problem

- Overfit to these two examples
- We need many more queries, covering more use cases